In [0]:
import sys
sys.path.append("/Workspace/SPTrans_v2.0")

from datetime import datetime
from pyspark.sql.functions import col, to_date, current_date
from core.config_loader import load_config
from core.sptrans_client import SPTransClient

config = load_config("/Workspace/SPTrans_v2.0/config.json")

sp = SPTransClient(
    base_url=config["sptrans"]["base_url"],
    token=config["sptrans"]["token"],
    proxies=config["sptrans"].get("proxy")
)

def coletar_paradas():
    if not sp.authenticate():
        print("❌ Falha ao autenticar")
        return

    # Lê a tabela de linhas
    linhas_df = spark.table("workspace.sptrans.bronze_linhas")

    # filtra apenas linhas do dia
    cod_linha_rows = (
        linhas_df
        .select("cod_linha")
        .where(
            (col("cod_linha").isNotNull()) &
            (to_date(col("fetch_time")) == current_date())
        )
        .distinct()
        .collect()
    )

    if not cod_linha_rows:
        print("⚠ Nenhum cod_linha encontrado. Rode coletar_linhas() antes.")
        return

    linhas = [int(r.cod_linha) for r in cod_linha_rows]
    print(f"✅ {len(linhas)} linhas com cod_linha encontradas para buscar paradas")

    registros = []
    ts_now = datetime.utcnow()

    for idx, cl in enumerate(linhas, start=1):
        try:
            print(f"[{idx}/{len(linhas)}] Buscando paradas da linha (cl={cl})")

            data = sp.get(f"Parada/BuscarParadasPorLinha?codigoLinha={cl}")

            if not data or not isinstance(data, list):
                print(f"ℹ️ Linha {cl} não possui paradas cadastradas.")
                continue

            for p in data:
                registros.append({
                    "fetch_time": ts_now,
                    "cod_linha": cl,
                    "cod_parada": p.get("cp"),
                    "nome_parada": p.get("np"),
                    "lat": p.get("py"),
                    "lng": p.get("px")
                })

        except Exception as e:
            print(f"❌ Erro ao buscar cl={cl}: {e}")

    if not registros:
        print("⚠ Nada retornado da API /Parada")
        return

    df = spark.createDataFrame(registros)

    df.write.format("delta") \
        .option("mergeSchema", "true") \
        .mode("overwrite") \
        .saveAsTable("workspace.sptrans.bronze_paradas")

    print(f"✅ {len(registros)} paradas salvas em bronze_paradas às {datetime.now().strftime('%H:%M:%S')}")
    display(df.limit(10))

coletar_paradas()


✅ Autenticado na API SPTrans
✅ 2286 linhas com cod_linha encontradas para buscar paradas
[1/2286] Buscando paradas da linha (cl=271)
ℹ️ Linha 271 não possui paradas cadastradas.
[2/2286] Buscando paradas da linha (cl=372)
ℹ️ Linha 372 não possui paradas cadastradas.
[3/2286] Buscando paradas da linha (cl=34849)
ℹ️ Linha 34849 não possui paradas cadastradas.
[4/2286] Buscando paradas da linha (cl=1768)
ℹ️ Linha 1768 não possui paradas cadastradas.
[5/2286] Buscando paradas da linha (cl=33027)
ℹ️ Linha 33027 não possui paradas cadastradas.
[6/2286] Buscando paradas da linha (cl=33170)
ℹ️ Linha 33170 não possui paradas cadastradas.
[7/2286] Buscando paradas da linha (cl=290)
ℹ️ Linha 290 não possui paradas cadastradas.
[8/2286] Buscando paradas da linha (cl=34374)
ℹ️ Linha 34374 não possui paradas cadastradas.
[9/2286] Buscando paradas da linha (cl=2630)
ℹ️ Linha 2630 não possui paradas cadastradas.
[10/2286] Buscando paradas da linha (cl=32909)
ℹ️ Linha 32909 não possui paradas cadastrad

cod_linha,cod_parada,fetch_time,lat,lng,nome_parada
32944,4203716,2025-11-09T23:48:38.869Z,-23.671866,-46.741665,PARADA PIRAPORINHA C/B
33214,614432,2025-11-09T23:48:38.869Z,-23.527608,-46.671511,BABY BARIONY C/B
33214,7014416,2025-11-09T23:48:38.869Z,-23.534377,-46.654322,ANGELICA C/B
33214,60014604,2025-11-09T23:48:38.869Z,-23.525262,-46.682254,SESC POMPEIA C/B
33214,60016783,2025-11-09T23:48:38.869Z,-23.525653,-46.679493,PARADA PALMEIRAS C/B
33214,60016785,2025-11-09T23:48:38.869Z,-23.52622,-46.674187,ANTARTICA C/B
33214,60016789,2025-11-09T23:48:38.869Z,-23.529636,-46.667777,PQ. DA ÁGUA BRANCA C/B
33214,480014608,2025-11-09T23:48:38.869Z,-23.522875,-46.688219,TIBERIO C/B
33214,480014609,2025-11-09T23:48:38.869Z,-23.522627,-46.692121,DUILIO C/B
33214,480014610,2025-11-09T23:48:38.869Z,-23.522339,-46.696246,TENDAL DA LAPA C/B


In [0]:
%sql
SELECT COUNT(*) AS total_registros
FROM workspace.sptrans.bronze_paradas;